# **Homework 8 - Topic Modeling**
# KDD Tuesdays 12:30 PM - 2:45 PM
## Jake Brulato

In [8]:
#import relvant packages for conduct topic modeling analysis
import os
import pandas as pd
import numpy as np
from pandas import DataFrame
np.random.seed(2023)

from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.corpora import Dictionary
from gensim.models import LdaMulticore

## **Defined Functions**

In [9]:
from nltk.stem import*
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

## **Read all the text files in the directory**

In [33]:
from nltk.corpus import PlaintextCorpusReader
corpus_root = 'MovieReviews' # Folder Name
file_names = PlaintextCorpusReader(corpus_root, '.*',encoding='latin-1')  # wildcard is read all files in the folder
file_names.fileids()  # Get the filenames

['16748.txt',
 '17108.txt',
 '17109.txt',
 '17110.txt',
 '17111.txt',
 '17116.txt',
 '17117.txt',
 '17118.txt',
 '17119.txt',
 '17139.txt',
 '17144.txt',
 '17145.txt',
 '17146.txt',
 '17147.txt',
 '17150.txt',
 '17185.txt',
 '17192.txt',
 '17219.txt',
 '17239.txt',
 '17243.txt',
 '17254.txt',
 '17255.txt',
 '17280.txt',
 '17300.txt',
 '17303.txt',
 '17341.txt',
 '17384.txt',
 '17391.txt',
 '17398.txt',
 '17399.txt',
 '17430.txt',
 '17431.txt',
 '17447.txt',
 '17457.txt',
 '17460.txt',
 '17501.txt',
 '17518.txt',
 '17532.txt',
 '17534.txt',
 '17578.txt',
 '17609.txt',
 '17610.txt',
 '17655.txt',
 '17662.txt',
 '17663.txt',
 '17695.txt',
 '17711.txt',
 '17713.txt',
 '17753.txt',
 '17757.txt',
 '17758.txt',
 '17761.txt',
 '17803.txt',
 '17811.txt',
 '17874.txt',
 '17879.txt',
 '17886.txt',
 '17896.txt',
 '17898.txt',
 '17902.txt',
 '17912.txt',
 '17933.txt',
 '17934.txt',
 '17945.txt',
 '17963.txt',
 '17971.txt',
 '17992.txt',
 '18004.txt',
 '18016.txt',
 '18032.txt',
 '18067.txt',
 '1806

## **Read all the text files in the directory**

In [34]:
# path: str = 'C:/Users/rbrul/Documents/GitHub/KDD/Homework_8/MovieReviews'
path: str = '/Users/jakebrulato/Documents/GitHub/KDD/Homework_8/MovieReviews'
file_names = os.listdir(path)

doc_contents: list = []
for i, file_name in zip(range(len(file_names)), file_names):
    with open(path + '/' + file_name, encoding="utf8", errors='ignore') as file:
        doc_contents.append((i, file_name, file.read()))
        
# for filename in os.listdir(path):
#     if filename.endswith(".txt"):
#         file_path = os.path.join(path, filename)
        
#         with open(file_path, 'r', encoding="latin-1") as file:
#             try:
#                 conetent = file.read()
#                 doc_contents.append({'Filename': filename, 'Content': conetent})
#             except UnicodeDecodeError:
#                 print(f"Error reading file {filename}")

# df = pd.DataFrame(doc_contents)

## **Build dictionary out of the document contents**

In [35]:
data = pd.DataFrame(doc_contents, columns=['RowNum', 'FileName', 'FileContent'])
data.dropna(subset=['FileContent'], inplace= True)
print(data)

# df



     RowNum   FileName                                        FileContent
0         0   1912.txt  Date Released: 5/14/93 Running Length: 1:51 Ra...
1         1   1866.txt  Joon (Mary Stuart Masterson) is a young woman ...
2         2   1867.txt  Starring: Lou Diamond Phillips, Jennifer Tilly...
3         3  18087.txt  LOVE IS THE DEVIL (director: John Maybury; Der...
4         4   1911.txt  Starring: Thomas Ian Griffith, Lance Henriksen...
..      ...        ...                                                ...
175     175  17145.txt  PEEPING TOM (director: Michael Powell; cast: C...
176     176  17384.txt  BOUCHER, LE (director: Claude Chabrol; St=E9ph...
177     177   2033.txt  THE LONG DAY CLOSES is a film that takes sever...
178     178   1937.txt  David Greenhill (Don Johnson) is about to go o...
179     179  17757.txt  SWEET HEREAFTER, THE (director: Atom Egoyan; c...

[180 rows x 3 columns]


## **Load documents contenets into a dataframe**

In [13]:
import gensim
import nltk
nltk.download('wordnet')

processed_docs = data['FileContent'].map(preprocess)
print(processed_docs[:5])

dictionary = Dictionary(processed_docs)
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jakebrulato/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0    [date, releas, run, length, rat, violenc, lang...
1    [joon, mari, stuart, masterson, young, woman, ...
2    [star, diamond, phillip, jennif, tilli, toshir...
3    [love, devil, director, john, mayburi, derek, ...
4    [star, thoma, griffith, lanc, henriksen, jam, ...
Name: FileContent, dtype: object


## **Build BagOfWords corpus from the document content**

In [14]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
print(len(bow_corpus))

180


## **Bag of Words test**

In [44]:
test_corpus = bow_corpus[0]
for i, word_stat in zip(range(len(test_corpus)), test_corpus):
    # print only the first 5 words
    if i < 5:
        print("Word {} (\"{}\") appears {} times.".format(word_stat[0], dictionary[word_stat[0]], word_stat[1]))

Word 0 ("action") appears 1 times.
Word 1 ("american") appears 3 times.
Word 2 ("arriv") appears 1 times.
Word 3 ("attempt") appears 2 times.
Word 4 ("awar") appears 1 times.


## **Build LDA model With 10 Topics**

In [45]:
lda_model = LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2, random_state=2023)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} -> Words: {}'.format(idx, topic))

Topic: 0 -> Words: 0.012*"school" + 0.007*"friend" + 0.007*"polic" + 0.006*"real" + 0.006*"best" + 0.006*"world" + 0.006*"wife" + 0.006*"problem" + 0.006*"great" + 0.006*"tri"
Topic: 1 -> Words: 0.016*"artist" + 0.007*"relationship" + 0.007*"turn" + 0.007*"perform" + 0.007*"paul" + 0.007*"see" + 0.006*"great" + 0.006*"imag" + 0.006*"woman" + 0.006*"audienc"
Topic: 2 -> Words: 0.013*"world" + 0.009*"girl" + 0.009*"show" + 0.008*"town" + 0.008*"famili" + 0.008*"feel" + 0.007*"group" + 0.007*"bring" + 0.007*"point" + 0.007*"father"
Topic: 3 -> Words: 0.009*"money" + 0.007*"wife" + 0.007*"action" + 0.007*"real" + 0.007*"screen" + 0.007*"littl" + 0.006*"great" + 0.006*"tri" + 0.006*"book" + 0.005*"role"
Topic: 4 -> Words: 0.011*"murder" + 0.009*"town" + 0.009*"intellectu" + 0.007*"citi" + 0.007*"audienc" + 0.006*"number" + 0.006*"tri" + 0.006*"experi" + 0.006*"see" + 0.006*"feel"
Topic: 5 -> Words: 0.017*"john" + 0.009*"happen" + 0.007*"help" + 0.006*"need" + 0.006*"action" + 0.006*"world" 

## **Top 10 Words for each topic**

In [46]:
top_topics = lda_model.top_topics(corpus=bow_corpus, topn=10)

i = 0
for words, coherence in top_topics:
    print('Topic: {} -> Top Words: {}'.format(i, words))
    i += 1

Topic: 0 -> Top Words: [(0.011950492, 'littl'), (0.007614736, 'music'), (0.007340963, 'releas'), (0.0070471624, 'audienc'), (0.0069833505, 'action'), (0.0066385916, 'perform'), (0.006119343, 'plot'), (0.005546241, 'role'), (0.0054461705, 'turn'), (0.005333835, 'best')]
Topic: 1 -> Top Words: [(0.013186485, 'feel'), (0.00937964, 'world'), (0.009287758, 'famili'), (0.007621972, 'see'), (0.00753347, 'father'), (0.0070649027, 'mother'), (0.0070038517, 'tri'), (0.0069729295, 'problem'), (0.0063099405, 'show'), (0.0060871714, 'need')]
Topic: 2 -> Top Words: [(0.0088611785, 'money'), (0.0074174036, 'wife'), (0.0072473236, 'action'), (0.0067129172, 'real'), (0.006621223, 'screen'), (0.0065465597, 'littl'), (0.0063300645, 'great'), (0.006078332, 'tri'), (0.005535863, 'book'), (0.0053430465, 'role')]
Topic: 3 -> Top Words: [(0.011910178, 'school'), (0.006874015, 'friend'), (0.006871782, 'polic'), (0.006473324, 'real'), (0.0063300235, 'best'), (0.0062303804, 'world'), (0.006136201, 'wife'), (0.00

## **Document -> All Topic probabilities for all the documents in the corpus**

In [51]:
topic_prob_per_doc = []

for doc in bow_corpus:
    topics = lda_model.get_document_topics(doc, minimum_probability=0.0)
    topic_prob_per_doc.append(topics)
    
for i, doc_topica in enumerate(topic_prob_per_doc):
    print(f"Document {i} :")
    for topic, prob in doc_topica:
        print(f"Filename: {data['FileName'][i]}, Topic {topic} with probability {prob}")

Document 0 :
Filename: 1912.txt, Topic 0 with probability 0.0008267093217000365
Filename: 1912.txt, Topic 1 with probability 0.0008266423246823251
Filename: 1912.txt, Topic 2 with probability 0.0008267764933407307
Filename: 1912.txt, Topic 3 with probability 0.058221809566020966
Filename: 1912.txt, Topic 4 with probability 0.0008267279481515288
Filename: 1912.txt, Topic 5 with probability 0.0008266884833574295
Filename: 1912.txt, Topic 6 with probability 0.0008267209050245583
Filename: 1912.txt, Topic 7 with probability 0.0008267303928732872
Filename: 1912.txt, Topic 8 with probability 0.7295746207237244
Filename: 1912.txt, Topic 9 with probability 0.2064165472984314
Document 1 :
Filename: 1866.txt, Topic 0 with probability 0.0007355365669354796
Filename: 1866.txt, Topic 1 with probability 0.0007355083944275975
Filename: 1866.txt, Topic 2 with probability 0.0007355134002864361
Filename: 1866.txt, Topic 3 with probability 0.04277509078383446
Filename: 1866.txt, Topic 4 with probability 

## **Document -> High probabilities for all the documents in the corpus**

In [18]:
for i, corpus_item in zip(range(len(bow_corpus)), bow_corpus):
    print(data['RowNum'][i], '-', data['FileName'][i], '->', lda_model[corpus_item])

0 - 1912.txt -> [(3, 0.070620775), (8, 0.7209387), (9, 0.20265356)]
1 - 1866.txt -> [(3, 0.052417517), (8, 0.9416983)]
2 - 1867.txt -> [(3, 0.63271797), (8, 0.35973254)]
3 - 18087.txt -> [(1, 0.9947961)]
4 - 1911.txt -> [(8, 0.99125963)]
5 - 1859.txt -> [(3, 0.026080782), (6, 0.18266924), (8, 0.785463)]
6 - 1865.txt -> [(3, 0.31791523), (8, 0.67606807)]
7 - 1864.txt -> [(3, 0.3439114), (8, 0.6512678)]
8 - 1858.txt -> [(3, 0.29105902), (8, 0.70283216)]
9 - 17610.txt -> [(7, 0.9935696)]
10 - 1910.txt -> [(6, 0.03644068), (8, 0.9579631)]
11 - 18480.txt -> [(7, 0.99347657)]
12 - 17758.txt -> [(0, 0.12545711), (3, 0.56763774), (7, 0.29997224)]
13 - 18447.txt -> [(7, 0.99613625)]
14 - 1928.txt -> [(8, 0.9940378)]
15 - 18080.txt -> [(7, 0.99268055)]
16 - 1860.txt -> [(8, 0.99342865)]
17 - 18282.txt -> [(2, 0.012264797), (6, 0.9145775), (7, 0.067173116)]
18 - 18283.txt -> [(2, 0.42379802), (7, 0.5689924)]
19 - 18136.txt -> [(0, 0.4216293), (6, 0.4217611), (7, 0.14967671)]
20 - 2005.txt -> [(8,